## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
#tf.enable_eager_execution()

TensorFlow 2.x selected.
2.0.0


In [0]:
import pandas as pd
import os
import numpy as np

In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [0]:
df_data = pd.read_csv("/content/drive/My Drive/PGPAIML/30Nov/Assignment/Internal Lab/prices.csv")
#print(os.getcwd())
#os.chdir
#!ls


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Check all columns in the dataset

In [6]:
df_data.head(10)

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0
5,2016-01-12 00:00:00,WLTW,115.510002,115.550003,114.500000,116.059998,1098000.0
6,2016-01-13 00:00:00,WLTW,116.459999,112.849998,112.589996,117.070000,949600.0
7,2016-01-14 00:00:00,WLTW,113.510002,114.379997,110.050003,115.029999,785300.0
8,2016-01-15 00:00:00,WLTW,113.330002,112.529999,111.919998,114.879997,1093700.0
9,2016-01-19 00:00:00,WLTW,113.660004,110.379997,109.870003,115.870003,1523500.0


### Drop columns `date` and  `symbol`

In [0]:
df_data.drop(['date','symbol'],inplace=True,axis=1)

In [8]:
df_data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
df_data['volume']=df_data['volume']/1000000

In [0]:
X = df_data.iloc[0:1000,0:4]
Y = df_data.iloc[0:1000,4]

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(X,Y,test_size=.30,random_state=2)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [23]:
train_y.shape

(700, 1)

In [24]:
train_y=train_y.reshape(-1,1)
train_y.shape

(700, 1)

In [0]:
W = tf.Variable( tf.zeros( [4,1] ) )
b = tf.Variable( tf.zeros( [1] ) ) 

2.Define a function to calculate prediction

In [0]:
def prediction(features, W, b):
    y_pred = tf.add(tf.matmul(features, W), b)
    return y_pred   

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(y_actual, y_predicted):
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
@tf.function
def train(x, y_actual, w, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    # Update Weights and Bias
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [34]:
 for i in range(100):
        
    W, b = train(train_x, train_y, W, b)
    if(((i+1) % 5) == 0):
      print('Current Training Loss on iteration', i+1, loss(train_y, prediction(train_x, W, b)))

('Current Training Loss on iteration', 5, <tf.Tensor: id=3111, shape=(), dtype=float32, numpy=218.64182>)
('Current Training Loss on iteration', 10, <tf.Tensor: id=3139, shape=(), dtype=float32, numpy=218.64177>)
('Current Training Loss on iteration', 15, <tf.Tensor: id=3167, shape=(), dtype=float32, numpy=218.64177>)
('Current Training Loss on iteration', 20, <tf.Tensor: id=3195, shape=(), dtype=float32, numpy=218.64172>)
('Current Training Loss on iteration', 25, <tf.Tensor: id=3223, shape=(), dtype=float32, numpy=218.64168>)
('Current Training Loss on iteration', 30, <tf.Tensor: id=3251, shape=(), dtype=float32, numpy=218.64165>)
('Current Training Loss on iteration', 35, <tf.Tensor: id=3279, shape=(), dtype=float32, numpy=218.6416>)
('Current Training Loss on iteration', 40, <tf.Tensor: id=3307, shape=(), dtype=float32, numpy=218.64156>)
('Current Training Loss on iteration', 45, <tf.Tensor: id=3335, shape=(), dtype=float32, numpy=218.64154>)
('Current Training Loss on iteration', 

### Get the shapes and values of W and b

In [35]:
W.shape

TensorShape([4, 1])

In [36]:
b.shape

TensorShape([1])

### Model Prediction on 1st Examples in Test Dataset

In [37]:
test_data=test_x[0]
test_data=test_data.reshape(-1,1).T
test_data.shape

(1, 4)

In [38]:
pred=prediction(test_x,W,B)
pred

<tf.Tensor: id=3647, shape=(300, 1), dtype=float32, numpy=
array([[2.8248909],
       [2.8243706],
       [2.8242996],
       [2.825814 ],
       [2.8244948],
       [2.8252857],
       [2.8241343],
       [2.8251898],
       [2.824259 ],
       [2.8242328],
       [2.8243105],
       [2.8249774],
       [2.825625 ],
       [2.8248186],
       [2.8244803],
       [2.8252778],
       [2.8241224],
       [2.8244107],
       [2.8250492],
       [2.8254807],
       [2.8242378],
       [2.8245995],
       [2.824054 ],
       [2.825395 ],
       [2.8242688],
       [2.8251748],
       [2.8246832],
       [2.8240924],
       [2.8244772],
       [2.8249285],
       [2.823864 ],
       [2.8241527],
       [2.8244133],
       [2.8245964],
       [2.8251529],
       [2.8244126],
       [2.8250694],
       [2.8247986],
       [2.8242457],
       [2.8253198],
       [2.8240511],
       [2.8253078],
       [2.824338 ],
       [2.8246338],
       [2.8244655],
       [2.8256261],
       [2.8243182],
 

## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
df_iris = pd.read_csv("/content/drive/My Drive/PGPAIML/30Nov/Assignment/Internal Lab/Iris.csv")

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [40]:
df_iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [41]:
df_iris = pd.get_dummies(df_iris, 'Species')
df_iris.head()



,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


### Splitting the data into feature set and target set

In [0]:
X = df_iris.iloc[:,1:5]
Y = df_iris.iloc[:,5:8]

In [95]:
X.shape

(150, 4)

In [0]:
trainX,testX,trainY,testY=train_test_split(X,Y,test_size=.30,random_state=2)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# Initialize Sequential model
model = tf.keras.models.Sequential()

input1 = tf.keras.layers.Input(shape=(4,))
model.add(input1)
# Normalize the data
model.add(tf.keras.layers.BatchNormalization())

# Add Dense Layer which provides 3 Outputs after applying softmax

model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [101]:
model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_16 (Batc (None, 4)                 16        
_________________________________________________________________
dense_22 (Dense)             (None, 3)                 15        
Total params: 31
Trainable params: 23
Non-trainable params: 8
_________________________________________________________________


### Model Training 

In [103]:
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=50,
          batch_size = trainX.shape[0])

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 0s 68us/sample - loss: 0.6888 - accuracy: 0.7143 - val_loss: 2.1052 - val_accuracy: 0.2889
Epoch 2/50
105/105 [==============================] - 0s 41us/sample - loss: 0.6859 - accuracy: 0.7238 - val_loss: 2.0903 - val_accuracy: 0.2889
Epoch 3/50
105/105 [==============================] - 0s 37us/sample - loss: 0.6831 - accuracy: 0.7238 - val_loss: 2.0754 - val_accuracy: 0.2889
Epoch 4/50
105/105 [==============================] - 0s 39us/sample - loss: 0.6803 - accuracy: 0.7238 - val_loss: 2.0605 - val_accuracy: 0.2889
Epoch 5/50
105/105 [==============================] - 0s 41us/sample - loss: 0.6776 - accuracy: 0.7238 - val_loss: 2.0454 - val_accuracy: 0.2889
Epoch 6/50
105/105 [==============================] - 0s 50us/sample - loss: 0.6749 - accuracy: 0.7238 - val_loss: 2.0303 - val_accuracy: 0.2889
Epoch 7/50
105/105 [==============================] - 0s 53us/sample - loss: 0.6723 -

### Model Prediction

In [113]:
model.predict(testX)

array([[0.3303123 , 0.04222519, 0.6274625 ],
       [0.3600695 , 0.06165596, 0.5782745 ],
       [0.01635248, 0.11305099, 0.87059647],
       [0.34960207, 0.06721651, 0.58318144],
       [0.29223326, 0.03934683, 0.66841996],
       [0.00491112, 0.02765631, 0.9674326 ],
       [0.2792096 , 0.06796949, 0.6528208 ],
       [0.0069948 , 0.07200424, 0.9210009 ],
       [0.00663643, 0.05350411, 0.9398594 ],
       [0.30291566, 0.07236017, 0.6247242 ],
       [0.20788583, 0.02360904, 0.7685051 ],
       [0.40955538, 0.05421542, 0.53622913],
       [0.28915218, 0.05194426, 0.65890354],
       [0.15045339, 0.01764045, 0.83190614],
       [0.01778916, 0.02742782, 0.954783  ],
       [0.01380543, 0.05480403, 0.9313905 ],
       [0.3631138 , 0.18188599, 0.45500016],
       [0.03727471, 0.06564384, 0.8970815 ],
       [0.00288973, 0.02224093, 0.9748693 ],
       [0.04825255, 0.12580419, 0.8259433 ],
       [0.00826376, 0.03649575, 0.95524055],
       [0.0193959 , 0.04995032, 0.9306538 ],
       [0.

### Save the Model

In [111]:
model.save('my_model.json')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: my_model.json/assets


### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?